# Создание таблицы и получение выборки данных
## Содержание
В этом уроке будет рассмотрен синтаксис и семантика следующих SQL запросов:

* создание таблиц и вставка значений;
* выборка всех данных из таблицы;
* выборка данных из отдельных столбцов;
* выборка отдельных столбцов и присвоение им новых имен;
* создание вычисляемых столбцов;
* вычисляемые столбцы, математические функции;
* вычисляемые столбцы, логические функции;
* выборка данных по простому условию;
* выборка данных с использованием логических выражений и операций;
* выборка данных, операторы BETWEEN, IN;
* выборка текстовых данных по шаблону, оператор LIKE;
* выборка данных с сортировкой.

Структура и наполнение таблицы

Все запросы будут формулироваться для таблицы book:

![table](./images/sql_intro_book_table.png)

## Создание таблицы и заполнение

In [1]:
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
config = {
        'user': 'root',
        'password': 'root',
        'host': 'db',
        'port': 3306,
        'database': 'SQLIntro',
        'charset': 'utf8'
    }

In [3]:
def create_table(query: str) -> None:
    connection = mysql.connector.connect(**config)
    cursor = connection.cursor()
    cursor.execute(query)
    cursor.close()
    connection.close()

In [4]:
def insert_values(query: str) -> None:
    connection = mysql.connector.connect(**config)
    cursor = connection.cursor()
    
    try:
        cursor.execute(query)
        connection.commit()
    except Exception as e:
        print(repr(e))
        connection.rollback()
    finally:
        cursor.close()
        connection.close()

In [5]:
# query = '''
#         CREATE TABLE book(
#                             book_id INT PRIMARY KEY AUTO_INCREMENT,
#                             title   VARCHAR(50),
#                             author  VARCHAR(30),
#                             price   DECIMAL(8, 2),
#                             amount  INT
#                          );
#         '''
# create_table(query)

In [6]:
# query = '''
#         INSERT INTO book (title, author, price, amount) VALUES ('Мастер и Маргарита', 'Булгаков М.А.', 670.99, 3),
#                                                                ('Белая гвардия', 'Булгаков М.А.', 540.50, 5),
#                                                                ('Идиот', 'Достоевский Ф.М.', 460.00, 10),
#                                                                ('Братья Карамазовы', 'Достоевский Ф.М.', 799.01, 2),
#                                                                ('Игрок', 'Достоевский Ф.М.', 480.50, 10),
#                                                                ('Стихотворения и поэмы', 'Есенин С.А.', 650.00, 15);

#         '''
# insert_values(query)

## Выборка данных

In [7]:
def select_data(query: str) -> pd.DataFrame:
    connection = mysql.connector.connect(**config)
    df_mysql = pd.read_sql(query, con=connection)
    connection.close()
    return df_mysql

In [8]:
query = '''
        SELECT *
        FROM book;
        '''

select_data(query).style.hide_index()

book_id,title,author,price,amount
1,Мастер и Маргарита,Булгаков М.А.,670.990000,3
2,Белая гвардия,Булгаков М.А.,540.500000,5
3,Идиот,Достоевский Ф.М.,460.000000,10
4,Братья Карамазовы,Достоевский Ф.М.,799.010000,2
5,Игрок,Достоевский Ф.М.,480.500000,10
6,Стихотворения и поэмы,Есенин С.А.,650.000000,15


In [9]:
query = '''
        SELECT author, title, price
        FROM book;
        '''

select_data(query).style.hide_index()

author,title,price
Булгаков М.А.,Мастер и Маргарита,670.990000
Булгаков М.А.,Белая гвардия,540.500000
Достоевский Ф.М.,Идиот,460.000000
Достоевский Ф.М.,Братья Карамазовы,799.010000
Достоевский Ф.М.,Игрок,480.500000
Есенин С.А.,Стихотворения и поэмы,650.000000


In [10]:
query = '''
        SELECT title as Название, author as Автор
        FROM book;
        '''

select_data(query).style.hide_index()

Название,Автор
Мастер и Маргарита,Булгаков М.А.
Белая гвардия,Булгаков М.А.
Идиот,Достоевский Ф.М.
Братья Карамазовы,Достоевский Ф.М.
Игрок,Достоевский Ф.М.
Стихотворения и поэмы,Есенин С.А.


In [11]:
query = '''
        SELECT title, amount, amount*1.65 AS pack
        FROM book;
        '''
select_data(query).style.hide_index()

title,amount,pack
Мастер и Маргарита,3,4.950000
Белая гвардия,5,8.250000
Идиот,10,16.500000
Братья Карамазовы,2,3.300000
Игрок,10,16.500000
Стихотворения и поэмы,15,24.750000


In [12]:
query = '''
        SELECT title, author, amount, round(price*0.7, 2) AS new_price
        FROM book;
        '''

select_data(query).style.hide_index()

title,author,amount,new_price
Мастер и Маргарита,Булгаков М.А.,3,469.690000
Белая гвардия,Булгаков М.А.,5,378.350000
Идиот,Достоевский Ф.М.,10,322.000000
Братья Карамазовы,Достоевский Ф.М.,2,559.310000
Игрок,Достоевский Ф.М.,10,336.350000
Стихотворения и поэмы,Есенин С.А.,15,455.000000


In [13]:
query = '''
        SELECT author, title,
               round(IF (author='Булгаков М.А.', price*1.10, IF(author='Есенин С.А.', price*1.05, price)), 2) AS new_price
        FROM book;
        '''

select_data(query).style.hide_index()

author,title,new_price
Булгаков М.А.,Мастер и Маргарита,738.090000
Булгаков М.А.,Белая гвардия,594.550000
Достоевский Ф.М.,Идиот,460.000000
Достоевский Ф.М.,Братья Карамазовы,799.010000
Достоевский Ф.М.,Игрок,480.500000
Есенин С.А.,Стихотворения и поэмы,682.500000


In [14]:
query = '''
        SELECT author, title, price
        FROM book
        WHERE amount < 10;
        '''

select_data(query).style.hide_index()

author,title,price
Булгаков М.А.,Мастер и Маргарита,670.990000
Булгаков М.А.,Белая гвардия,540.500000
Достоевский Ф.М.,Братья Карамазовы,799.010000


In [15]:
query = '''
        SELECT title, author, price, amount
        FROM book
        WHERE price*amount >= 5000 and (price < 500 or price > 600);
        '''

select_data(query).style.hide_index()

title,author,price,amount
Стихотворения и поэмы,Есенин С.А.,650.000000,15


In [16]:
query = '''
        SELECT title, author
        FROM book
        WHERE (price BETWEEN 540.50 and 800) and    amount IN (2, 3, 5, 7);
        '''

select_data(query).style.hide_index()

title,author
Мастер и Маргарита,Булгаков М.А.
Белая гвардия,Булгаков М.А.
Братья Карамазовы,Достоевский Ф.М.


In [17]:
query = '''
        SELECT title, author
        FROM book
        WHERE title LIKE '% %' AND author LIKE '%С.%';
        '''

select_data(query).style.hide_index()

title,author
Стихотворения и поэмы,Есенин С.А.


In [18]:
query = '''
        SELECT author, title
        FROM book
        WHERE amount BETWEEN 2 AND 14 
        ORDER BY author DESC, title ASC;
        '''

select_data(query).style.hide_index()

author,title
Достоевский Ф.М.,Братья Карамазовы
Достоевский Ф.М.,Игрок
Достоевский Ф.М.,Идиот
Булгаков М.А.,Белая гвардия
Булгаков М.А.,Мастер и Маргарита


## Групповые операции

In [19]:
query = '''
        SELECT DISTINCT amount
        FROM book;
        '''

select_data(query).style.hide_index()

amount
3
5
10
2
15


In [20]:
query = '''
        SELECT author AS 'Автор', COUNT(author) AS 'Различных_книг', SUM(amount) AS 'Количество_экземпляров'
        FROM book
        GROUP BY author;
        '''

select_data(query).style.hide_index()

Автор,Различных_книг,Количество_экземпляров
Булгаков М.А.,2,8.000000
Достоевский Ф.М.,3,22.000000
Есенин С.А.,1,15.000000


In [21]:
query = '''
        SELECT author, MIN(price) AS 'Минимальная цена', MAX(price) AS 'Максимальная цена', AVG(price) AS 'Средняя цена'
        FROM book
        GROUP BY author;
        '''

select_data(query).style.hide_index()

author,Минимальная цена,Максимальная цена,Средняя цена
Булгаков М.А.,540.500000,670.990000,605.745000
Достоевский Ф.М.,460.000000,799.010000,579.836667
Есенин С.А.,650.000000,650.000000,650.000000


In [22]:
query = '''
        SELECT author, round(SUM(price*amount), 2) AS 'Стоимость', round(SUM(price*amount) * 0.18 / (1 + 0.18), 2) AS 'НДС',
               round(SUM(price*amount) / (1 + 0.18), 2) AS 'Стоимость_без_НДС'
        FROM book
        GROUP BY author;
        '''

select_data(query).style.hide_index()

author,Стоимость,НДС,Стоимость_без_НДС
Булгаков М.А.,4715.470000,719.310000,3996.160000
Достоевский Ф.М.,11003.020000,1678.430000,9324.590000
Есенин С.А.,9750.000000,1487.290000,8262.710000


In [23]:
query = '''
        SELECT MIN(price) AS 'Минимальная_цена', MAX(price) AS 'Максимальная_цена', ROUND(AVG(price), 2) AS 'Средняя_цена'
        FROM book;
        '''

select_data(query).style.hide_index()

Минимальная_цена,Максимальная_цена,Средняя_цена
460.000000,799.010000,600.170000


In [24]:
query = '''
        SELECT ROUND(AVG(price), 2) AS 'Средняя_цена', ROUND(SUM(price*amount), 2) AS 'Стоимость'
        FROM book
        WHERE amount BETWEEN 5 and 14;
        '''

select_data(query).style.hide_index()

Средняя_цена,Стоимость
493.670000,12107.500000


In [25]:
query = '''
        SELECT author, SUM(price*amount) AS 'Стоимость'
        FROM book
        WHERE title <> 'Идиот' and title <> 'Белая гвардия'
        GROUP BY author
        HAVING Стоимость > 5000
        ORDER BY Стоимость DESC;
        '''

select_data(query).style.hide_index()

author,Стоимость
Есенин С.А.,9750.000000
Достоевский Ф.М.,6403.020000


## Вложенные запросы

In [26]:
query = '''
        SELECT author, title, price
        FROM book
        WHERE price <= (SELECT AVG(price) FROM book)
        ORDER BY price DESC;
        '''

select_data(query).style.hide_index()

author,title,price
Булгаков М.А.,Белая гвардия,540.500000
Достоевский Ф.М.,Игрок,480.500000
Достоевский Ф.М.,Идиот,460.000000


In [27]:
query = '''
        SELECT author, title, price
        FROM book
        WHERE price - (SELECT MIN(price) FROM book) <= 150
        ORDER BY price;
        '''

select_data(query).style.hide_index()

author,title,price
Достоевский Ф.М.,Идиот,460.000000
Достоевский Ф.М.,Игрок,480.500000
Булгаков М.А.,Белая гвардия,540.500000


In [28]:
query = '''
        SELECT author, title, amount
        FROM book
        WHERE amount IN (SELECT amount FROM book GROUP BY amount HAVING COUNT(amount) = 1);
        '''

select_data(query).style.hide_index()

author,title,amount
Булгаков М.А.,Мастер и Маргарита,3
Булгаков М.А.,Белая гвардия,5
Достоевский Ф.М.,Братья Карамазовы,2
Есенин С.А.,Стихотворения и поэмы,15


In [29]:
query = '''
        SELECT author, title, price
        FROM book
        WHERE author = ANY (SELECT author FROM book GROUP BY author HAVING AVG(price) > (SELECT AVG(price) FROM book));
        '''

select_data(query).style.hide_index()

author,title,price
Булгаков М.А.,Мастер и Маргарита,670.990000
Булгаков М.А.,Белая гвардия,540.500000
Есенин С.А.,Стихотворения и поэмы,650.000000


In [30]:
query = '''
        SELECT title, author, amount, (SELECT MAX(amount) FROM book) - amount AS 'Заказ'
        FROM book
        WHERE amount <> (SELECT MAX(amount) FROM book);
        '''

select_data(query).style.hide_index()

title,author,amount,Заказ
Мастер и Маргарита,Булгаков М.А.,3,12
Белая гвардия,Булгаков М.А.,5,10
Идиот,Достоевский Ф.М.,10,5
Братья Карамазовы,Достоевский Ф.М.,2,13
Игрок,Достоевский Ф.М.,10,5


In [31]:
query = '''
        SELECT title, author, amount, (SELECT MAX(amount) FROM book) - amount AS 'Заказ'
        FROM book
        WHERE amount <> (SELECT MAX(amount) FROM book);
        '''

select_data(query).style.hide_index()

title,author,amount,Заказ
Мастер и Маргарита,Булгаков М.А.,3,12
Белая гвардия,Булгаков М.А.,5,10
Идиот,Достоевский Ф.М.,10,5
Братья Карамазовы,Достоевский Ф.М.,2,13
Игрок,Достоевский Ф.М.,10,5


## Запросы корректировки данных

In [32]:
# query = '''
#         CREATE TABLE supply(supply_id INT PRIMARY KEY AUTO_INCREMENT,
#                             title VARCHAR(50),
#                             author VARCHAR(30),
#                             price DECIMAL(8, 2),
#                             amount INT);
#         '''
# create_table(query)

In [33]:
# query = '''
#         INSERT INTO supply (title, author, price, amount) VALUES ('Лирика', 'Пастернак Б.Л.', 518.99, 2),
#                                                                  ('Черный человек', 'Есенин С.А.', 570.20, 6),
#                                                                  ('Белая гвардия', 'Булгаков М.А.', 540.50, 7),
#                                                                  ('Идиот', 'Достоевский Ф.М.', 360.80, 3);
#         '''
# insert_values(query)

In [34]:
query = '''
        SELECT title, author, price, amount
        FROM supply
        '''

select_data(query).style.hide_index()

title,author,price,amount
Лирика,Пастернак Б.Л.,518.990000,2
Черный человек,Есенин С.А.,570.200000,6
Белая гвардия,Булгаков М.А.,540.500000,7
Идиот,Достоевский Ф.М.,360.800000,3


In [35]:
query = '''
        INSERT INTO book (title, author, price, amount)
            SELECT title, author, price, amount
            FROM supply
            WHERE author NOT IN ('Булгаков М.А.', 'Достоевский Ф.М.');
        '''

insert_values(query)

In [36]:
query = '''
        SELECT title, author, price, amount
        FROM book
        '''

select_data(query).style.hide_index()

title,author,price,amount
Мастер и Маргарита,Булгаков М.А.,670.990000,3
Белая гвардия,Булгаков М.А.,540.500000,5
Идиот,Достоевский Ф.М.,460.000000,10
Братья Карамазовы,Достоевский Ф.М.,799.010000,2
Игрок,Достоевский Ф.М.,480.500000,10
Стихотворения и поэмы,Есенин С.А.,650.000000,15
Лирика,Пастернак Б.Л.,518.990000,2
Черный человек,Есенин С.А.,570.200000,6


In [37]:
query = '''
        INSERT INTO book (title, author, price, amount)
               SELECT title, author, price, amount
               FROM supply
               WHERE author NOT IN (SELECT author FROM book);
        '''

insert_values(query)

In [38]:
query = '''
        SELECT title, author, price, amount
        FROM book
        '''

select_data(query).style.hide_index()

title,author,price,amount
Мастер и Маргарита,Булгаков М.А.,670.990000,3
Белая гвардия,Булгаков М.А.,540.500000,5
Идиот,Достоевский Ф.М.,460.000000,10
Братья Карамазовы,Достоевский Ф.М.,799.010000,2
Игрок,Достоевский Ф.М.,480.500000,10
Стихотворения и поэмы,Есенин С.А.,650.000000,15
Лирика,Пастернак Б.Л.,518.990000,2
Черный человек,Есенин С.А.,570.200000,6


In [39]:
query = '''
        UPDATE book SET price = 0.9 * price
                    WHERE amount BETWEEN 5 and 10;
        '''

insert_values(query)

In [40]:
query = '''
        SELECT title, author, price, amount
        FROM book
        '''

select_data(query).style.hide_index()

title,author,price,amount
Мастер и Маргарита,Булгаков М.А.,670.990000,3
Белая гвардия,Булгаков М.А.,486.450000,5
Идиот,Достоевский Ф.М.,414.000000,10
Братья Карамазовы,Достоевский Ф.М.,799.010000,2
Игрок,Достоевский Ф.М.,432.450000,10
Стихотворения и поэмы,Есенин С.А.,650.000000,15
Лирика,Пастернак Б.Л.,518.990000,2
Черный человек,Есенин С.А.,513.180000,6


In [41]:

query = '''
        ALTER TABLE book
        ADD COLUMN buy INT DEFAULT 0;
        '''

insert_values(query)

In [42]:
query = '''
        UPDATE book SET buy = IF(title = 'Белая гвардия', 3, IF(title = 'Идиот', 8, IF(title = 'Стихотворения и поэмы', 15, buy)));
        '''

insert_values(query)

In [43]:
query = '''
        UPDATE book SET buy = IF(buy <= amount, buy, amount);
        '''

insert_values(query)

In [44]:
query = '''
        SELECT title, author, price, amount, buy
        FROM book
        '''

select_data(query).style.hide_index()

title,author,price,amount,buy
Мастер и Маргарита,Булгаков М.А.,670.990000,3,0
Белая гвардия,Булгаков М.А.,486.450000,5,3
Идиот,Достоевский Ф.М.,414.000000,10,8
Братья Карамазовы,Достоевский Ф.М.,799.010000,2,0
Игрок,Достоевский Ф.М.,432.450000,10,0
Стихотворения и поэмы,Есенин С.А.,650.000000,15,15
Лирика,Пастернак Б.Л.,518.990000,2,0
Черный человек,Есенин С.А.,513.180000,6,0


In [45]:
query = '''
        UPDATE book, supply SET book.amount = book.amount + supply.amount,
                                book.price = IF(book.price <> supply.price, round((book.price + supply.price) / 2, 2), book.price)
        WHERE book.title = supply.title and book.author = supply.author;
        '''

insert_values(query)

In [46]:
query = '''
        SELECT title, author, price, amount, buy
        FROM book
        '''

select_data(query).style.hide_index()

title,author,price,amount,buy
Мастер и Маргарита,Булгаков М.А.,670.990000,3,0
Белая гвардия,Булгаков М.А.,513.480000,12,3
Идиот,Достоевский Ф.М.,387.400000,13,8
Братья Карамазовы,Достоевский Ф.М.,799.010000,2,0
Игрок,Достоевский Ф.М.,432.450000,10,0
Стихотворения и поэмы,Есенин С.А.,650.000000,15,15
Лирика,Пастернак Б.Л.,518.990000,4,0
Черный человек,Есенин С.А.,541.690000,12,0


In [47]:
query = '''
        DELETE FROM supply
        WHERE title IN (SELECT title FROM book WHERE book.price = supply.price);
        '''

insert_values(query)

In [49]:
query = '''
        SELECT title, author, price, amount
        FROM supply
        '''

select_data(query).style.hide_index()

title,author,price,amount
Черный человек,Есенин С.А.,570.200000,6
Белая гвардия,Булгаков М.А.,540.500000,7
Идиот,Достоевский Ф.М.,360.800000,3


In [51]:
query = '''
        CREATE TABLE ordering AS
                SELECT author, title, 
                       (SELECT round(AVG(amount)) FROM book) AS amount
                FROM book
                WHERE amount < (SELECT round(AVG(amount)) FROM book);
        '''

create_table(query)

In [52]:
query = '''
        SELECT *
        FROM ordering;
        '''

select_data(query).style.hide_index()

author,title,amount
Булгаков М.А.,Мастер и Маргарита,9.000000
Достоевский Ф.М.,Братья Карамазовы,9.000000
Пастернак Б.Л.,Лирика,9.000000


## Таблица "Командировки", запросы на выборку

In [53]:
# query = '''
#         CREATE TABLE trip (
#                              trip_id INT PRIMARY KEY AUTO_INCREMENT, 
#                              name VARCHAR(30), 
#                              city VARCHAR(25), 
#                              per_diem DECIMAL (8,2), 
#                              date_first DATE, 
#                              date_last DATE 
#                           );
#         '''

# create_table(query)

In [56]:
# query = '''
#         INSERT INTO trip (name, city, per_diem, date_first, date_last)
#         VALUES ('Абрамова К.А.', 'Владивосток', 450.00, STR_TO_DATE('2020-07-02', '%Y-%m-%d'), STR_TO_DATE('2020-07-13', '%Y-%m-%d')),
#                ('Федорова А.Ю.', 'Томск', 450.00, STR_TO_DATE('2020-06-20', '%Y-%m-%d'), STR_TO_DATE('2020-06-26', '%Y-%m-%d')),
#                ('Абрамова К.А.', 'Санкт-Петербург', 700.00, STR_TO_DATE('2020-05-28', '%Y-%m-%d'), STR_TO_DATE('2020-06-04', '%Y-%m-%d')),
#                ('Федорова А.Ю.', 'Новосибирск', 450.00, STR_TO_DATE('2020-05-25', '%Y-%m-%d'), STR_TO_DATE('2020-06-04', '%Y-%m-%d')),
#                ('Абрамова К.А.', 'Москва', 700.00, STR_TO_DATE('2020-04-06', '%Y-%m-%d'), STR_TO_DATE('2020-04-14', '%Y-%m-%d')),
#                ('Абрамова К.А.', 'Москва', 700.00, STR_TO_DATE('2020-02-23', '%Y-%m-%d'), STR_TO_DATE('2020-03-01', '%Y-%m-%d')),
#                ('Абрамова К.А.', 'Владивосток', 450.00, STR_TO_DATE('2020-01-14', '%Y-%m-%d'), STR_TO_DATE('2020-01-27', '%Y-%m-%d'));
#         '''

# insert_values(query)

In [57]:
query = '''
        SELECT name, city, per_diem, date_first, date_last
        FROM trip
        WHERE name LIKE '%а %'
        ORDER BY date_last DESC;
        '''

select_data(query).style.hide_index()

name,city,per_diem,date_first,date_last
Абрамова К.А.,Владивосток,450.000000,2020-07-02,2020-07-13
Федорова А.Ю.,Томск,450.000000,2020-06-20,2020-06-26
Абрамова К.А.,Санкт-Петербург,700.000000,2020-05-28,2020-06-04
Федорова А.Ю.,Новосибирск,450.000000,2020-05-25,2020-06-04
Абрамова К.А.,Москва,700.000000,2020-04-06,2020-04-14
Абрамова К.А.,Москва,700.000000,2020-02-23,2020-03-01
Абрамова К.А.,Владивосток,450.000000,2020-01-14,2020-01-27
